In [42]:
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dot, Flatten
from sklearn.model_selection import train_test_split

In [4]:
data_project = pd.read_csv('generate_random_data_project.csv',names=["project_id", "project_category"])
freelancer_preference = pd.read_csv('new_combined_preferences.csv')

In [6]:
result_data = []

# Loop melalui setiap baris di freelancer_preference
for _, row in freelancer_preference.iterrows():
    freelancer_id = row['freelancer_id']
    
    # Loop melalui setiap kolom kecuali 'id_freelancer'
    for col in freelancer_preference.columns[1:]:
        project_category = col
        preference_freelancer = row[col]
        
        # Filter data_project berdasarkan game_type
        filtered_data_project = data_project[data_project['project_category'] == project_category]
        
        # Loop melalui setiap baris di data_project yang sesuai
        for _, project_row in filtered_data_project.iterrows():
            project_id = project_row['project_id']
            
            # Tambahkan data ke list
            result_data.append({'project_id': project_id, 'freelancer_id': freelancer_id, 'project_category': project_category, 'preference_freelancer': preference_freelancer})

In [7]:
result = pd.DataFrame(result_data)

In [8]:
print(result)

      project_id  freelancer_id project_category  preference_freelancer
0              1              1             Moba                      2
1             14              1             Moba                      2
2             21              1             Moba                      2
3             31              1             Moba                      2
4             53              1             Moba                      2
...          ...            ...              ...                    ...
44795        610             70        Adventure                      5
44796        613             70        Adventure                      5
44797        615             70        Adventure                      5
44798        621             70        Adventure                      5
44799        629             70        Adventure                      5

[44800 rows x 4 columns]


In [9]:
result.to_csv('new_merged_data.csv', index=False)

In [98]:
#TRAIN MODEL STEP

projects = pd.read_csv('original_data_project.csv')
preferences = pd.read_csv('new_merged_data.csv')

In [99]:
n_project = preferences.project_id.nunique()
n_freelancer = preferences.freelancer_id.nunique()

In [16]:
train, test = train_test_split(preferences, test_size=0.2)

In [74]:
EMBEDDING_DIM = 27

#input layer
project_input = Input(shape=1)
freelancer_input = Input(shape=1)

#embedding layer
project_embedding = Embedding(n_project+1, EMBEDDING_DIM)(project_input)
freelancer_embedding = Embedding(n_freelancer+1, EMBEDDING_DIM)(freelancer_input)

#flatten layer
project_flat = Flatten()(project_embedding)
freelancer_flat = Flatten()(freelancer_embedding)

#output layer
output = Dot(1)([project_flat, freelancer_flat])

model = Model([project_input, freelancer_input],[output])

In [75]:
model.compile(optimizer='adam', loss='mse')

In [76]:
model.fit(x=[train.project_id, train.freelancer_id], y=[train.preference_freelancer],
         epochs=30, batch_size=128)

Epoch 1/30
280/280 [==============================] - 2s 4ms/step - loss: 14.5790
Epoch 2/30
280/280 [==============================] - 1s 3ms/step - loss: 5.4438
Epoch 3/30
280/280 [==============================] - 1s 3ms/step - loss: 1.3691
Epoch 4/30
280/280 [==============================] - 1s 4ms/step - loss: 1.2535
Epoch 5/30
280/280 [==============================] - 1s 3ms/step - loss: 1.2302
Epoch 6/30
280/280 [==============================] - 1s 3ms/step - loss: 1.1825
Epoch 7/30
280/280 [==============================] - 1s 3ms/step - loss: 1.1167
Epoch 8/30
280/280 [==============================] - 1s 3ms/step - loss: 1.0543
Epoch 9/30
280/280 [==============================] - 1s 3ms/step - loss: 1.0091
Epoch 10/30
280/280 [==============================] - 1s 3ms/step - loss: 0.9815
Epoch 11/30
280/280 [==============================] - 1s 3ms/step - loss: 0.9654
Epoch 12/30
280/280 [==============================] - 1s 3ms/step - loss: 0.9543
Epoch 13/30
280/280 [===

In [77]:
model.evaluate(x=[test.project_id, test.freelancer_id], y=[test.preference_freelancer])

280/280 [==============================] - 1s 3ms/step - loss: 0.0305


0.03053835779428482

In [78]:
predictions = model.predict([test.project_id, test.freelancer_id])

280/280 [==============================] - 1s 3ms/step


In [79]:
for i in range(10):
    print(f"Actual Rating: {test.preference_freelancer.iloc[i]}, Predicted Rating: {predictions[i][0]}")

Actual Rating: 2, Predicted Rating: 1.998559832572937
Actual Rating: 3, Predicted Rating: 3.1291935443878174
Actual Rating: 3, Predicted Rating: 2.9185824394226074
Actual Rating: 5, Predicted Rating: 4.932645320892334
Actual Rating: 5, Predicted Rating: 4.995343208312988
Actual Rating: 3, Predicted Rating: 3.208770513534546
Actual Rating: 3, Predicted Rating: 3.0705626010894775
Actual Rating: 5, Predicted Rating: 4.96804666519165
Actual Rating: 5, Predicted Rating: 4.814556121826172
Actual Rating: 2, Predicted Rating: 2.013911008834839


In [80]:
# import numpy as np
# import matplotlib.pyplot as plt

# def get_recommendations(id_freelancer, projects, model):
#     projects = projects.copy()
#     id_freelancers = np.array([id_freelancer] * len(projects))
#     results = model([projects.id_project.values, id_freelancers]).numpy().reshape(-1)

#     projects['predicted_rating'] = pd.Series(results)
#     projects = projects.sort_values('predicted_rating', ascending=False)

#     print(f'Recommendations for user {id_freelancer}')
#     return(projects)

# get_recommendations(100, projects, model)

In [104]:
import numpy as np
import tensorflow as tf
import pandas as pd

def get_recommendations(freelancer_id, projects, model):
    try:
        projects = projects.copy()
        id_freelancers = np.array([freelancer_id] * len(projects))
        results = model([projects.project_id.values, id_freelancers]).numpy().reshape(-1)

        projects['predicted_rating'] = pd.Series(results)
        projects = projects.sort_values('predicted_rating', ascending=False)

        print(f'Recommendations for user {freelancer_id}')
        return projects
    except tf.errors.InvalidArgumentError:
        print(f'User {freelancer_id} not found. Returning random recommendations.')
        # Menggunakan np.random.randint untuk mendapatkan nilai random_state yang berbeda setiap kali dijalankan
        random_state = np.random.randint(1, 100)
        projects = projects.sample(n=10, random_state=random_state)
        return projects

# Contoh pemanggilan fungsi
get_recommendations(1, projects, model)


Recommendations for user 1


,id,project_id,project_name,project_desc,user_id,deadline,project_category,imgUrl,predicted_rating
76,77,77,Supernatural Horror Game Companion,Menemani pemain selama bermain game horor supr...,7,2023-12-25T00:00:00Z,Horor,https://i.imgur.com/5y377j3.jpg,4.818808
209,210,210,Chess Endgame Tactics Tutor,Tutor untuk membantu pemain menguasai taktik e...,9,2023-12-25T00:00:00Z,Puzzle,https://i.imgur.com/1815742.jpg,4.806592
92,93,93,Zombie Apocalypse Survival Guide,Panduan bertahan hidup dan strategi dalam meng...,7,2023-12-25T00:00:00Z,Horor,https://i.imgur.com/5y377j3.jpg,4.803967
123,124,124,Open World Wildlife Photography Tour,Tur fotografi satwa liar dalam dunia terbuka game,8,2023-12-30T00:00:00Z,OpenWorld,https://i.imgur.com/5y377j3.jpg,4.800283
469,470,470,Open World Photography Expedition in Ghost of ...,Menyelenggarakan ekspedisi fotografi dalam dun...,8,2023-12-30T00:00:00Z,OpenWorld,https://i.imgur.com/1815742.jpg,4.796750
...,...,...,...,...,...,...,...,...,...
260,261,261,RPG Party Leader Strategy Guide Final Fantasy XIV,Memberikan panduan strategi kepada pemimpin ke...,3,2023-12-25T00:00:00Z,RPG,https://i.imgur.com/5y377j3.jpg,1.819618
60,61,61,Virtual Fitness Coach in Ring Fit Adventure,Berkolaborasi untuk mencapai tujuan kebugaran ...,17,2023-12-25T00:00:00Z,Adventure,https://i.imgur.com/5y377j3.jpg,1.765268
113,114,114,Horror VR Escape Room Experience,Pengalaman escape room horor dalam virtual rea...,7,2023-12-25T00:00:00Z,Horor,https://i.imgur.com/5y377j3.jpg,1.756682
454,455,455,Tournament Level Coaching in Hearthstone,Memberikan pelatihan untuk mencapai tingkat tu...,13,2023-12-30T00:00:00Z,Puzzle,https://i.imgur.com/5y377j3.jpg,1.731862


In [19]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 embedding (Embedding)       (None, 1, 30)                38430     ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, 1, 30)                2130      ['input_2[0][0]']             
                                                                                              

In [81]:
model.save('recommendermodel.h5')

c:\users\crozi\appdata\local\programs\python\python39\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
